## 透過這次訓練所學到的東西

#### 特殊的方法:
#### 先用KNN預測labels為0或1 (沒病,有病)
#### 再用KNN的predict_proba預測兩者的機率 (0.8,0.2)
#### submission有病的機率(0.2)

In [454]:
# 正規化對準確度是否有影響?

# Normalization(正規化)相關係數 和 準確率都沒變

# 第一次 ['thal']欄位字串轉為int就直接上傳 4.60081

## 引入套件

In [260]:
import pandas as pd

In [261]:
# 載入資料
train_values = pd.read_csv("train_values.csv") 
train_labels = pd.read_csv("train_labels.csv")

In [262]:
print(train_values.shape)
train_values.head()

(180, 14)


,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,0z64un,1,normal,128,2,0,0,2,308,0.0,1,45,170,0
1,ryoo3j,2,normal,110,3,0,0,0,214,1.6,0,54,158,0
2,yt1s1x,1,normal,125,4,3,0,2,304,0.0,1,77,162,1
3,l2xjde,1,reversible_defect,152,4,0,0,0,223,0.0,1,40,181,0
4,oyt4ek,3,reversible_defect,178,1,0,0,2,270,4.2,1,59,145,0


In [263]:
print(train_labels.shape)
train_labels.head()

(180, 2)


,patient_id,heart_disease_present
0,0z64un,0
1,ryoo3j,0
2,yt1s1x,1
3,l2xjde,1
4,oyt4ek,0


In [264]:
# 將字串轉為int,不一定要加 .astype(int)  結果還是會是int64
train_values['thal'] = train_values['thal'].map({'normal':0, 'fixed_defect':1, 'reversible_defect':2})
train_values.head()
# train_values.dtypes  看型態

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,0z64un,1,0,128,2,0,0,2,308,0.0,1,45,170,0
1,ryoo3j,2,0,110,3,0,0,0,214,1.6,0,54,158,0
2,yt1s1x,1,0,125,4,3,0,2,304,0.0,1,77,162,1
3,l2xjde,1,2,152,4,0,0,0,223,0.0,1,40,181,0
4,oyt4ek,3,2,178,1,0,0,2,270,4.2,1,59,145,0


In [265]:
# # 計算各欄位,可用info()取代
# train_values.count()
train_values.info()   # 看資料摘要資訊,共有180row,發現沒有空值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 14 columns):
patient_id                              180 non-null object
slope_of_peak_exercise_st_segment       180 non-null int64
thal                                    180 non-null int64
resting_blood_pressure                  180 non-null int64
chest_pain_type                         180 non-null int64
num_major_vessels                       180 non-null int64
fasting_blood_sugar_gt_120_mg_per_dl    180 non-null int64
resting_ekg_results                     180 non-null int64
serum_cholesterol_mg_per_dl             180 non-null int64
oldpeak_eq_st_depression                180 non-null float64
sex                                     180 non-null int64
age                                     180 non-null int64
max_heart_rate_achieved                 180 non-null int64
exercise_induced_angina                 180 non-null int64
dtypes: float64(1), int64(12), object(1)
memory usage: 19.8+ KB


In [266]:
train_values.describe()  # 查詢統計資訊

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
count,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,1.550000,0.866667,131.311111,3.155556,0.694444,0.161111,1.050000,249.211111,1.010000,0.688889,54.811111,149.483333,0.316667
std,0.618838,0.971091,17.010443,0.938454,0.969347,0.368659,0.998742,52.717969,1.121357,0.464239,9.334737,22.063513,0.466474
min,1.000000,0.000000,94.000000,1.000000,0.000000,0.000000,0.000000,126.000000,0.000000,0.000000,29.000000,96.000000,0.000000
25%,1.000000,0.000000,120.000000,3.000000,0.000000,0.000000,0.000000,213.750000,0.000000,0.000000,48.000000,132.000000,0.000000
50%,1.000000,0.000000,130.000000,3.000000,0.000000,0.000000,2.000000,245.500000,0.800000,1.000000,55.000000,152.000000,0.000000
75%,2.000000,2.000000,140.000000,4.000000,1.000000,0.000000,2.000000,281.250000,1.600000,1.000000,62.000000,166.250000,1.000000
max,3.000000,2.000000,180.000000,4.000000,3.000000,1.000000,2.000000,564.000000,6.200000,1.000000,77.000000,202.000000,1.000000


In [267]:
# 去除離群值


In [268]:
# # Normalization(正規化)將資料縮放在0~1之間    相關係數沒有改變,為什麼還需要正規化?      
# # 有網址也把Normalization叫(標準化)?  

# # 相關係數沒有改變,為什麼還需要正規化?   參考(https://hk.saowen.com/a/ee8f4a328e55f10a3c97d2c8688d4fc79c3fd933e3d95bffed947ec8402cc1fe)
# # 1. 提升模型的收斂速度
# # 2. 提升模型的精度

# # Standardization(標準化) 是什麼???

# # 方式1 : 相關係數沒有變,都是0.079775
# # train['serum_cholesterol_mg_per_dl'] = train['serum_cholesterol_mg_per_dl'] / train['serum_cholesterol_mg_per_dl'].max()
# # train['serum_cholesterol_mg_per_dl'].head()

# # 方式2 : 相關係數沒有變,都是0.079775,準確率也沒變  (浮點數值跟方法一不太一樣)
# train_values['serum_cholesterol_mg_per_dl'] = (train_values['serum_cholesterol_mg_per_dl']-train_values['serum_cholesterol_mg_per_dl'].min())/(train_values['serum_cholesterol_mg_per_dl'].max()-train_values['serum_cholesterol_mg_per_dl'].min())
# train_values['serum_cholesterol_mg_per_dl'].head()

## 分析相關係數

In [269]:
# 因為要分析相關係數，所以將label併到feature
# train_labels['heart_disease_present'] = sj_train_labels.total_cases
train = pd.merge(train_values,train_labels,on='patient_id',how='inner')
# 計算所有欄位跟label的相關係數
correlation = train.corr()
correlation.loc['heart_disease_present'].sort_values(ascending=False)

heart_disease_present                   1.000000
thal                                    0.538780
exercise_induced_angina                 0.448647
num_major_vessels                       0.421519
chest_pain_type                         0.412829
oldpeak_eq_st_depression                0.382930
slope_of_peak_exercise_st_segment       0.344224
sex                                     0.335421
resting_ekg_results                     0.145933
age                                     0.138255
serum_cholesterol_mg_per_dl             0.079775
resting_blood_pressure                  0.078506
fasting_blood_sugar_gt_120_mg_per_dl    0.003379
max_heart_rate_achieved                -0.375352
Name: heart_disease_present, dtype: float64

## 選擇要分析的欄位

In [270]:
# 選相關係數0.4以上  # 0.38796759570602446
values = train_values[["thal","exercise_induced_angina","num_major_vessels","chest_pain_type"]]

# # 選相關係數前7名  # 
# values = train_values[["thal","exercise_induced_angina","num_major_vessels","chest_pain_type","oldpeak_eq_st_depression","slope_of_peak_exercise_st_segment","sex"]]

# # 選相關係數前3名  # 0.43115157413619165
# values = train_values[["thal","exercise_induced_angina","num_major_vessels"]]

# # 選相關係數前5名  # 0.3296344134119075
# values = train_values[["thal","exercise_induced_angina","num_major_vessels","chest_pain_type","oldpeak_eq_st_depression"]]

# # 選相關係數前6名  # 0.3042835536259936
# values = train_values[["thal","exercise_induced_angina","num_major_vessels","chest_pain_type","oldpeak_eq_st_depression","slope_of_peak_exercise_st_segment"]]

labels = train_labels[['heart_disease_present']]

## 資料拆分(train、test)

In [271]:
import numpy as np
# 引入機器學習演算法
from sklearn.neighbors import KNeighborsClassifier   # KNN
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC   #支援向量機  
from sklearn import tree  # 決策樹
from sklearn.ensemble import RandomForestClassifier  # Random Forest

# 引入切割資料的功能
from sklearn.model_selection import train_test_split
# 引入儲存model的功能
from sklearn.externals import joblib

# # 資料前處理
# from sklearn.preprocessing import StandardScaler
# # 降維
# from sklearn.decomposition import PCA

In [272]:
# 0.2 -> 0.38750475366970377   0.3 -> 0.3946748203027913    0.1 -> 0.39861886311772476
# 資料拆分  ## 記得 y=答案  test=驗證
# 學習特徵, 驗證特徵, 學習答案, 驗證答案 = 資料切割(原始特徵,原始答案, 保留2成資料去驗證, 亂數種子)
x_train, x_test, y_train, y_test = train_test_split(values, labels, test_size=0.2, random_state=70)

# random_state    13-> 0.38750475366970377->34名     
# 2-> 0.36787999857800674->42名       4-> 0.36510956389486043->42名    17-> 0.3568597934912722
# 21-> 0.3791680754943363   22-> 0.3749800262898549   26-> 0.3627385321436985   28-> 0.3598068955737664  30-> 0.33335365861153726->38名
# 31-> 0.3672452542722275   32-> 0.37157508908824055   33-> 0.3701359048574233    40-> 0.3385742590365678->38名
# 48-> 0.3396027237947587->38名     65-> 0.3614049496317117-> 40名    70-> 0.33404814315164777->31名

## 選擇模型

In [273]:
# clf = LogisticRegression() # 0.4052582011162684
# clf.fit(x_train, y_train)

# # KNN     0.9755720534102053
# clf = KNeighborsClassifier(n_neighbors=4)
# clf.fit(x_train, y_train) 

# 支援向量機  0.38750475366970377
clf = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=100, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf.fit(x_train, y_train)  # 學習特徵, 學習答案

# # 決策樹   0.25450144259133267  但提交上去的成績為2.80984,連低於1都還沒達到
# clf = tree.DecisionTreeClassifier()
# clf.fit(x_train, y_train) 

# # 隨機森林   0.43698443637204115
# clf = RandomForestClassifier(max_depth=2, random_state=0)
# clf.fit(x_train, y_train)  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=100, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [274]:
clf.predict(x_test)

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0], dtype=int64)

In [275]:
# array([[沒有心臟病, 有心臟病],[沒有心臟病, 有心臟病]....[沒有心臟病, 有心臟病]])
clf.predict_proba(x_test)

array([[0.87611432, 0.12388568],
       [0.91049891, 0.08950109],
       [0.87611432, 0.12388568],
       [0.07590935, 0.92409065],
       [0.91049891, 0.08950109],
       [0.9239188 , 0.0760812 ],
       [0.8761796 , 0.1238204 ],
       [0.89769205, 0.10230795],
       [0.9239188 , 0.0760812 ],
       [0.10132102, 0.89867898],
       [0.86976751, 0.13023249],
       [0.88314708, 0.11685292],
       [0.13203724, 0.86796276],
       [0.06275437, 0.93724563],
       [0.87611432, 0.12388568],
       [0.87600568, 0.12399432],
       [0.87603608, 0.12396392],
       [0.06275437, 0.93724563],
       [0.9239188 , 0.0760812 ],
       [0.06275437, 0.93724563],
       [0.51383792, 0.48616208],
       [0.91049891, 0.08950109],
       [0.11607518, 0.88392482],
       [0.14890942, 0.85109058],
       [0.87611432, 0.12388568],
       [0.87600568, 0.12399432],
       [0.87615178, 0.12384822],
       [0.06275437, 0.93724563],
       [0.75223937, 0.24776063],
       [0.13203724, 0.86796276],
       [0.

In [276]:
from sklearn.metrics import log_loss

in_sample_preds = clf.predict_proba(x_train)
# log_loss(真實值，預測值)
log_loss(y_train, in_sample_preds)

0.3341213865647453

In [277]:
# log_loss(y_train, in_sample_preds)

In [278]:
# # 儲存建立的機器模型
# joblib.dump(svm, "lr_model",compress=9)

## 預測test_values

In [279]:
test_values = pd.read_csv("test_values.csv")
# test_values.shape
# test_values.info()

# 將字串轉為int,不一定要加 .astype(int)  結果還是會是int64
test_values['thal'] = test_values['thal'].map({'normal':0, 'fixed_defect':1, 'reversible_defect':2})
# 選擇要分析的欄位
test = test_values[["thal","exercise_induced_angina","num_major_vessels","chest_pain_type"]]
# clf.predict_proba()的值為[0.83031932, 0.16968068] --> [沒有心臟病, 有心臟病]  [:, 1](取有心臟病的機率)
predictions = clf.predict_proba(test)[:, 1]

## 寫法一

In [280]:
#  讀取submission_format
submission_format = pd.read_csv('submission_format.csv', index_col='patient_id')
output = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

## 寫法二

In [281]:
# #  讀取submission_format
# submission_format = pd.read_csv("submission_format.csv")
# # 刪掉原本答案的欄位
# submission_format = submission_format.drop(['heart_disease_present'], axis=1)
# # 合併submission_format跟test_predictions
# x = pd.DataFrame(data=predictions)
# output = pd.concat([submission_format,x], axis=1)

In [282]:
output.head()

,heart_disease_present
patient_id,
olalu7,0.123908
z9n6mx,0.123886
5k4413,0.867963
mrg7q5,0.123994
uki4do,0.646422


In [283]:
# 輸出csv檔
output.to_csv('output0812_6.csv')

In [122]:
# # 將預測結果 numpy轉pandas  (1維度轉成2維度,才能合併)
# #                               轉成2維度後的欄位名稱 : 預測的結果(1維度)
# test_predictions = pd.DataFrame({"heart_disease_present": test_predictions})